In [1]:
import os
import sys
sys.path.append("..")

In [2]:
from mmengine import Config
from global_config import *

## Modifiable Configuration

In [3]:
FINAL_MODEL_CONFIG_DIR = CONFIG_BISENETV2_FCN_DIR_PATH
WORK_DIR = OUT_BISENETV2_FCN_DIR_PATH
MODEL_CFG_NAME = "bisenetv2_fcn"
mmseg_model_cfg_suffix_path = "bisenetv2/bisenetv2_fcn_4xb8-160k_cityscapes-1024x1024.py"
# mmseg_model_cfg_path = Path(MMSEG_MODEL_CONFIG_DIR_PATH)/Path(mmseg_model_cfg_subdir)
mmseg_model_cfg_suffix_path

'bisenetv2/bisenetv2_fcn_4xb8-160k_cityscapes-1024x1024.py'

In [4]:
mmseg_model_cfg_path = os.path.join(MMSEG_MODEL_CONFIG_DIR_PATH, mmseg_model_cfg_suffix_path)
print(mmseg_model_cfg_path)

cfg = Config.fromfile(mmseg_model_cfg_path)
celeba_pipeline_cfg = Config.fromfile(CELEBA_PIPELINE_PATH)
cfg.merge_from_dict(celeba_pipeline_cfg)


/home/featurize/work/AI6126project1/mmsegmentation/configs/bisenetv2/bisenetv2_fcn_4xb8-160k_cityscapes-1024x1024.py


In [5]:
# # Modifiable
cfg.model.backbone.norm_cfg = NORM_CFG
cfg.norm_cfg = NORM_CFG 
cfg.data_preprocessor = DATA_PREPROCESSOR
cfg.model.data_preprocessor = DATA_PREPROCESSOR

for i in range(len(cfg.model.auxiliary_head)):
    cfg.model.auxiliary_head[i].norm_cfg = NORM_CFG
    cfg.model.auxiliary_head[i].num_classes = NUM_CLASS
    
cfg.model.decode_head.num_classes = NUM_CLASS

cfg.work_dir = WORK_DIR
cfg.train_dataloader.batch_size = 8

cfg.train_cfg.max_iters = 30000
cfg.train_cfg.val_interval = 500 
cfg.default_hooks.logger.interval = 100 
cfg.default_hooks.checkpoint.interval = 500
cfg.default_hooks.checkpoint.max_keep_ckpts = 2
cfg.default_hooks.checkpoint.save_best = 'mIoU'
cfg.default_hooks.checkpoint.rule = "greater"

cfg['randomness'] = dict(seed=0)


In [6]:
cfg.model.decode_head.loss_decode = weighted_CE_lovasz_dice_loss_decode
# for i in range(len(cfg.model.auxiliary_head)):
#     cfg.model.auxiliary_head[i].loss_decode = weighted_CE_lovasz_dice_loss_decode_auxiliary

for i in range(len(cfg.model.auxiliary_head)):
    cfg.model.auxiliary_head[i].loss_decode.class_weight = class_weights
    cfg.model.auxiliary_head[i].loss_decode.loss_weight *= 0.4

In [7]:
#xs 1.7M
cfg.model.backbone.detail_channels = (32, 64, 96)
cfg.model.backbone.semantic_channels = (16, 32, 64, 96)
cfg.model.backbone.bga_channels = 96
cfg.model.backbone.out_indices = (0,1,2,3,4)
cfg.model.backbone.semantic_expansion_ratio = 6

cfg.model.decode_head.in_channels = 96
cfg.model.decode_head.channels = 512
cfg.model.decode_head.in_index = 0

cfg.model.auxiliary_head[0].in_channels = 16
cfg.model.auxiliary_head[0].channels = 16
cfg.model.auxiliary_head[0].in_index = 1
cfg.model.auxiliary_head[1].in_channels = 32
cfg.model.auxiliary_head[1].channels = 32
cfg.model.auxiliary_head[1].in_index = 2
cfg.model.auxiliary_head[2].in_channels = 64
cfg.model.auxiliary_head[2].channels = 64
cfg.model.auxiliary_head[2].in_index = 3
cfg.model.auxiliary_head[3].in_channels = 96
cfg.model.auxiliary_head[3].channels = 128
cfg.model.auxiliary_head[3].in_index = 4

In [8]:
cfg.optimizer = dict(lr=0.0075, momentum=0.9, weight_decay=0.001, type='SGD')
cfg.optim_wrapper = dict(type='OptimWrapper', optimizer=cfg.optimizer)
cfg.param_scheduler = [
    dict(begin=0, by_epoch=False, end=1000, start_factor=0.2, type='LinearLR'), 
    dict(
        begin=1000,
        by_epoch=False,
        end=cfg.train_cfg.max_iters,   
        eta_min=5e-5,  
        power=0.95,
        type='PolyLR'),
]

In [9]:
# #s 2.2M
# cfg.model.backbone.detail_channels = (32, 64, 128)
# cfg.model.backbone.semantic_channels = (16, 32, 64, 128)
# cfg.model.backbone.bga_channels = 128
# cfg.model.backbone.out_indices = (0,1,2,3)
# cfg.model.backbone.semantic_expansion_ratio = 6

# cfg.model.decode_head.in_channels = 128
# cfg.model.decode_head.channels = 128
# cfg.model.decode_head.in_index = 0

# cfg.model.auxiliary_head[0].in_channels = 16
# cfg.model.auxiliary_head[0].channels = 16
# cfg.model.auxiliary_head[0].in_index = 1
# cfg.model.auxiliary_head[1].in_channels = 32
# cfg.model.auxiliary_head[1].channels = 32
# cfg.model.auxiliary_head[1].in_index = 2
# cfg.model.auxiliary_head[2].in_channels = 64
# cfg.model.auxiliary_head[2].channels = 64
# cfg.model.auxiliary_head[2].in_index = 3

# del cfg.model.auxiliary_head[3]

# # cfg.model.auxiliary_head[3].in_channels = 96
# # cfg.model.auxiliary_head[3].channels = 128
# # cfg.model.auxiliary_head[3].in_index = 4

In [10]:
#print(mmseg_model_cfg.pretty_text)

In [11]:
from datetime import datetime
ymd_timestamp = datetime.now().strftime("%Y%m%d")
cfg.dump(os.path.join(FINAL_MODEL_CONFIG_DIR, f"celeba_{MODEL_CFG_NAME}_{ymd_timestamp}.py"))
os.path.join(FINAL_MODEL_CONFIG_DIR, f"celeba_{MODEL_CFG_NAME}_{ymd_timestamp}.py")

'/home/featurize/work/AI6126project1/assets/model_configs/bisenetv2_fcn/celeba_bisenetv2_fcn_20250327.py'